In [2]:
import pandas as pd
import re

In [3]:
emails = pd.read_csv("../../source-data/emails.csv")

In [6]:
def extract_data():
    df = pd.DataFrame(columns=['date',
                               'sender_address',
                               'recipient',
                               'subject',
                               'sender_name',
                               'recipient_name',
                               'cc',
                               'bcc',
                               'folder',
                               'body'])
    
    for index, details in emails.iterrows():
        raw_email_info = details['message']
        # raw_email_info = f_in.read().decode('utf8')
        date = re.findall(r'Date: (.*)', raw_email_info)[0]
        sender_address = re.findall(r'From: (.*)', raw_email_info)[0]
        recipient = re.findall(r'To: (.*)', raw_email_info)[0]
        subject = re.findall(r'Subject: (.*)', raw_email_info)[0]
        sender_name = re.findall(r'X-From: (.*)', raw_email_info)[0]
        recipient_name = re.findall(r'X-To: ([ A-Za-z]*)', raw_email_info)[0]
        cc = re.findall(r'X-cc: (.*)', raw_email_info)[0]
        bcc = re.findall(r'X-bcc: (.*)', raw_email_info)[0]
        folder = re.findall(r'"[a-zA-z-]*/(.*)/.*,"Message-ID.*>', raw_email_info)
        
        # strip everything before X-FileName
        bodies = re.findall(r'(?<=X-FileName: )(?s)(.*$)', raw_email_info)
        # Get everything after the first newline
        bodies = bodies[0].split('\n')
        body = " ".join(bodies[1:])
        
        df.loc[index] = [date, sender_address, recipient, subject, sender_name, recipient_name, cc, bcc, folder, body]
        
    return df
                         
m = extract_data()

KeyboardInterrupt: 

In [ ]:
print(m.shape)

## Cleaning the text

In [33]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from collections import Counter
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim import corpora, models
import gensim
import numpy as np

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
texts = []

In [37]:
for i in m['body']:
    #print "Processing",i
    # clean and tokenize document string
    tokens = tokenizer.tokenize(i)
    # remove all numbers
    tokens = [x for x in tokens if not (x.isdigit() or x[0] == '-' and x[1:].isdigit())]
    # remove structural words
    tokens = [x for x in tokens if len(x) > 1]
    tokens = [x.lower() for x in tokens]
    tokens = [x for x in tokens if 'http' not in x]
    tokens = [x for x in tokens if x not in "_"]
    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    # add tokens to list
    texts.append(stemmed_tokens)

dictionaryall = corpora.Dictionary(texts)

corpusall = [dictionaryall.doc2bow(text) for text in texts]

texts[1]

['travel',
 'busi',
 'meet',
 'take',
 'fun',
 'trip',
 'especi',
 'prepar',
 'present',
 'suggest',
 'hold',
 'busi',
 'plan',
 'meet',
 'take',
 'trip',
 'without',
 'formal',
 'busi',
 'meet',
 'even',
 'tri',
 'get',
 'honest',
 'opinion',
 'whether',
 'trip',
 'even',
 'desir',
 'necessari',
 'far',
 'busi',
 'meet',
 'think',
 'product',
 'tri',
 'stimul',
 'discuss',
 'across',
 'differ',
 'group',
 'work',
 'often',
 'present',
 'speak',
 'other',
 'quiet',
 'just',
 'wait',
 'turn',
 'meet',
 'might',
 'better',
 'held',
 'round',
 'tabl',
 'discuss',
 'format',
 'suggest',
 'go',
 'austin',
 'play',
 'golf',
 'rent',
 'ski',
 'boat',
 'jet',
 'ski',
 'fli',
 'somewher',
 'take',
 'much',
 'time']

In [43]:
texts[4]

['let', 'shoot', 'tuesday']

In [40]:
ldamodelall = gensim.models.ldamodel.LdaModel(corpusall, num_topics=7, id2word = dictionaryall, passes=20,
                                              minimum_probability=0)

/Users/rutika/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:497: RuntimeWarning: overflow encountered in exp
  expElogthetad = np.exp(Elogthetad)


In [39]:
print(ldamodelall.print_topics(num_topics=5, num_words=5))

[(4, 'nan*"direct" + nan*"determin" + nan*"document" + nan*"difficult" + nan*"down"'), (3, 'nan*"direct" + nan*"determin" + nan*"document" + nan*"difficult" + nan*"down"'), (2, 'nan*"direct" + nan*"determin" + nan*"document" + nan*"difficult" + nan*"down"'), (0, 'nan*"direct" + nan*"determin" + nan*"document" + nan*"difficult" + nan*"down"'), (6, 'nan*"direct" + nan*"determin" + nan*"document" + nan*"difficult" + nan*"down"')]
